In [11]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import time

frames_folder = 'images/train'
empty_parking_folder = 'final_base'
output = 'output'

In [4]:
def perspective_transform(image, src_points, dst_points):
    matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    warped = cv2.warpPerspective(image, matrix, (image.shape[1], image.shape[0]))
    return warped

In [5]:
def apply_best_blur(image, method="gaussian", kernel_size=3, sigma=0, diameter=9, sigma_color=200, sigma_space=200):
    if method == "gaussian":
        blurred_image = cv2.GaussianBlur(image, (kernel_size, kernel_size), sigma)
    elif method == "median":
        blurred_image = cv2.medianBlur(image, kernel_size)
    elif method == "bilateral":
        blurred_image = cv2.bilateralFilter(image, diameter, sigma_color, sigma_space)
    else:
        raise ValueError("incorrect method")
    
    return blurred_image

In [6]:
def show_in_row(list_of_images, titles = None, disable_ticks = False):
    count = len(list_of_images)
    for idx in range(count):
        subplot = plt.subplot(1, count, idx+1)
        if titles is not None:
            subplot.set_title(titles[idx])

        img = list_of_images[idx]
        cmap = 'gray' if (len(img.shape) == 2 or img.shape[2] == 1) else None
        subplot.imshow(img, cmap=cmap)
        if disable_ticks:
            plt.xticks([]), plt.yticks([])
    plt.show()

In [22]:
def mse_match(imageA, imageB):
    err = np.mean((imageA.astype("float") - imageB.astype("float")) ** 2)
    
    return err <= 1200.0

def images_match(img1, img2, method=cv2.TM_CCOEFF_NORMED, threshold=0.8):
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)

    res = cv2.matchTemplate(img2_gray, img1_gray, method)
    min_val, max_val, _, _ = cv2.minMaxLoc(res)

    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
        match_value = min_val
        # For these methods, lower values indicate a better match
        match = match_value <= (1 - threshold)
    else:
        match_value = max_val
        # For these methods, higher values indicate a better match
        match = match_value >= threshold

    return match

def images_match_ncc(img1, img2, threshold=0.2):
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)

    ncc = np.corrcoef(img2_gray, img1_gray)

    return ncc[0, -1] >= threshold

In [ ]:
parking_spaces = np.load('parking_spaces.npy')

In [ ]:
def main():
    template_spaces = np.float32([
        [[2, 0], [17, 0], [1, 22], [17, 22]],
        [[8, 2], [34, 3], [1, 43], [26, 44]],
    ])

    template_files = sorted([f for f in os.listdir(empty_parking_folder) if f.endswith('.jpg') or f.endswith('.png')])
    templates = []
    for i, template_file in enumerate(template_files):
        template_path = os.path.join(empty_parking_folder, template_file)
        template_image = cv2.imread(template_path)    
        template_image = cv2.cvtColor(template_image, cv2.COLOR_BGR2RGB)
        template_image = cv2.GaussianBlur(template_image, (3,3), 0)

        x, y, _ = template_image.shape
        
        dst_points = np.float32([[0, 0], [y, 0], [0, x], [y, x]])

        new_image = perspective_transform(template_image, template_spaces[i], dst_points)
        new_image = cv2.resize(new_image, (50, 80), interpolation=cv2.INTER_CUBIC)
        
        templates.append(new_image)

    frame_files = sorted([f for f in os.listdir(frames_folder) if f.endswith('.jpg') or f.endswith('.png')])

    for idx, frame_file in enumerate(frame_files):

        if idx == 100:
            break
        idx += 1
        
        frame_path = os.path.join(frames_folder, frame_file)
        frame = cv2.imread(frame_path)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        for idx, template in enumerate(templates):
            x, y, _ = template_image.shape
            dst_points = np.float32([[0, 0], [y, 0], [0, x], [y, x]])
                
            for parking_place_coor in parking_spaces:
                frame_copy = perspective_transform(frame, parking_place_coor, dst_points)[0:x, 0:y]
                frame_copy = cv2.resize(frame_copy, (50, 80), interpolation=cv2.INTER_CUBIC)
                
                if mse_match(frame_copy, template):
                    parking_place_coor[[0, 1, 2, 3]] = parking_place_coor[[0, 1, 3, 2]]
                    pts = np.array(parking_place_coor, np.int32)
                    parking_place_coor[[0, 1, 2, 3]] = parking_place_coor[[0, 1, 3, 2]]
                    pts = pts.reshape((-1,1,2))
                    cv2.polylines(frame,[pts],True,(255,0,0), 2)
                
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        cv2.imwrite(output + '/' + frame_file, frame)

In [77]:
if __name__ == "__main__":
    start_time = time.time()
    main()
    print(f"Working time: {time.time() - start_time} sec.")

Working time: 22.255393266677856 sec.
